In [ ]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.0 MB/s 
     |████████████████████████████████| 4.4 MB 60.9 MB/s 
     |████████████████████████████████| 212 kB 74.5 MB/s 
     |████████████████████████████████| 1.1 MB 44.0 MB/s 
     |████████████████████████████████| 140 kB 66.7 MB/s 
     |████████████████████████████████| 101 kB 12.1 MB/s 
     |████████████████████████████████| 596 kB 66.0 MB/s 
     |████████████████████████████████| 127 kB 69.6 MB/s 
     |████████████████████████████████| 6.6 MB 50.9 MB/s 
     |████████████████████████████████| 271 kB 71.8 MB/s 
     |████████████████████████████████| 144 kB 74.0 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing

In [ ]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,text
0,"Mkhedruli became more and more dominant over the two other scripts , though Khutsuri ( Nuskhuri with Asomtavruli ) was used until the 19th century . Since the 19th century , with the establishment and development of the printed Georgian fonts , Mkhedruli became universal writing Georgian outside the Church . \n"
1,Kawin Lari ( Elope ; 1975 ) \n
2,
3,"The poet 's intention to recall the Homeric character remains evident in certain passages . "" I am become a name "" ( 11 ) recalls an episode in the Odyssey in which Demodocus sings about Odysseus ' adventures in the king 's presence , acknowledging his fame . With phrases such as "" There gloom the dark broad seas "" ( 45 ) and "" The deep / Moans round with many voices "" ( 55 – 56 ) , Tennyson seems to be consciously invoking Homer . \n"
4,"The prose play Edwards Dream ( Eduards Traum ) was released in 1891 , composed of several small grouped episodes , rather than one linear storyline . The work received mixed reception . Joseph Kraus felt it was the peak of Busch 's life 's work , his nephews called it a masterwork of world literature , and the publisher of a critical collective edition spoke of a narrative style that is not found in contemporary literature . Eva Weissweiler saw in the play Busch 's attempt to prove himself in the novella genre , believing that everything that angered or insulted him , and his accompanying emotional depths , are apparent in the story . The 1895 story The Butterfly ( Der Schmetterling ) parodies themes and motifs and ridicules the religious optimism of a German romanticism which contradicted Busch 's realistic anthropology influenced by Schopenhauer and Charles Darwin . Its prose is more stringent in narrative style compared to Edwards Dream . Both were not popular amongst readers , because of their unfamiliar style . \n"
5,"In 1950 Alan Turing published a landmark paper in which he speculated about the possibility of creating machines that think . He noted that "" thinking "" is difficult to define and devised his famous Turing Test . If a machine could carry on a conversation ( over a teleprinter ) that was indistinguishable from a conversation with a human being , then it was reasonable to say that the machine was "" thinking "" . This simplified version of the problem allowed Turing to argue convincingly that a "" thinking machine "" was at least plausible and the paper answered all the most common objections to the proposition . The Turing Test was the first serious proposal in the philosophy of artificial intelligence . \n"
6,
7,= = = = Special effects = = = = \n
8,
9,


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base", use_fast=True)

In [ ]:
def tokenize_function(examples):
  return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (544 > 512). Running this sequence through the model will result in indexing errors


...

Token indices sequence length is longer than the specified maximum sequence length for this model (560 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (638 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
block_size = 128


def group_texts(examples):
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  total_length = len(concatenated_examples[list(examples.keys())[0]])
  total_length = (total_length // block_size) * block_size
  result = {
    k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
    for k, t in concatenated_examples.items()
  }
  result["labels"] = result["input_ids"].copy()
  return result

In [ ]:
lm_datasets = tokenized_datasets.map(
  group_texts,
  batched=True,
  batch_size=1000,
  num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/10 [00:00<?, ?ba/s]

...

#2:   0%|          | 0/10 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForMaskedLM, TrainingArguments, Trainer
model = AutoModelForMaskedLM.from_pretrained("distilroberta-base")

In [ ]:
training_args = TrainingArguments(
  "distilroberta-base-finetuned",
  evaluation_strategy = "epoch",
  learning_rate=2e-5,
  weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=lm_datasets["train"],
  eval_dataset=lm_datasets["validation"],
  data_collator=data_collator,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 19242
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7218


Epoch,Training Loss,Validation Loss
1,2.084200,1.921928
2,1.991300,1.882243
3,1.959600,1.821514


Saving model checkpoint to distilroberta-base-finetuned/checkpoint-500
Configuration saved in distilroberta-base-finetuned/checkpoint-500/config.json
Model weights saved in distilroberta-base-finetuned/checkpoint-500/pytorch_model.bin
Saving model checkpoint to distilroberta-base-finetuned/checkpoint-1000
Configuration saved in distilroberta-base-finetuned/checkpoint-1000/config.json
Model weights saved in distilroberta-base-finetuned/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to distilroberta-base-finetuned/checkpoint-1500
Configuration saved in distilroberta-base-finetuned/checkpoint-1500/config.json
Model weights saved in distilroberta-base-finetuned/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to distilroberta-base-finetuned/checkpoint-2000
Configuration saved in distilroberta-base-finetuned/checkpoint-2000/config.json
Model weights saved in distilroberta-base-finetuned/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 198

TrainOutput(global_step=7218, training_loss=2.0394436610838356, metrics={'train_runtime': 1406.726, 'train_samples_per_second': 41.036, 'train_steps_per_second': 5.131, 'total_flos': 1913931601832448.0, 'train_loss': 2.0394436610838356, 'epoch': 3.0})